厳密対角化のモジュール

In [1]:
module ExactDiag
using LinearAlgebra
using PrettyTables
using SparseArrays
using UnionFind
using Printf
const _dim = Ref(1)
const _site = Ref(1)
const _trans = Ref(Vector{Vector{Int}}())
const _reverse = Ref(Dict{Vector{Int},Int}())
export _dim, _site, init, Op, id, shift, spin_flip, site_flip, spin, num, S_z, matrix, show_eigens, energy_same_check, block_diag1, block_diag1_rvalue, block_diag1_entanglement_entropy# ここに外でも使う関数を列挙する
function init(dim::Int, site::Int)
  _dim[] = dim
  _site[] = site
  dim_tot = dim^site
  _trans[] = Vector{Vector{UInt8}}(undef, dim_tot)
  row = zeros(Int, site)
  _trans[][1] = copy(row)
  @inbounds for i in 1:dim_tot-1
    j = site
    while true
      v = row[j] + 1
      if v == dim
        row[j] = 0
        j -= 1
      else
        row[j] = v
        break
      end
    end
    _trans[][i+1] = copy(row)
  end
  _reverse[] = Dict{Vector{Int},Int}(_trans[][i] => i for i in eachindex(_trans[]))
end
function Nary_reverse(n::Vector{Int})
  if haskey(_reverse[], n)
    return copy(_reverse[][n])
  elseif length(n) != _site[]
    throw(ArgumentError("n must be a vector of length site"))
  else
    throw(ArgumentError("n is not a valid state"))
  end
end
function Nary_trans(t::Int)
  if t == 0
    return [-1 for _ in 1:_site[]]
  elseif t < 1 || t > _dim[]^_site[]
    throw(ArgumentError("t must be in the range [0, dim^site - 1]"))
  else
    return copy(_trans[][t])
  end
end
struct Op
  op::Vector{Tuple{ComplexF64,Vector{Function}}}
  function Op(op::Vector{Tuple{ComplexF64,Vector{Function}}})
    new(op)
  end
  function Op(op1::Function)
    vecf = Vector{Function}([op1])
    Op([(1.0 + 0.0im, vecf)])
  end
end
import Base: *, +, -, show, sum
function +(ops::Op...)
  k = Vector{Tuple{ComplexF64,Vector{Function}}}()
  for op1 in ops
    k = vcat(k, op1.op)
  end
  Op(k)
end
function *(op1::Op)
  op1
end
function *(op1::Op, op2::Op...)
  op3 = *(op2...)
  k = Vector{Tuple{ComplexF64,Vector{Function}}}()
  for op11 in op1.op
    for op31 in op3.op
      push!(k, (op11[1] * op31[1], vcat(op11[2], op31[2])))
    end
  end
  Op(k)
end
function *(coeff::Union{ComplexF64,Float64}, op1::Op...)
  op2 = *(op1...)
  k = Vector{Tuple{ComplexF64,Vector{Function}}}()
  for op21 in op2.op
    push!(k, (op21[1] * ComplexF64(coeff), op21[2]))
  end
  Op(k)
end
function *(op1::Op, t::Int)
  t1 = copy(t)
  for op11 in op1.op[1][2]
    t1 = op11(t1)[2]
  end
  sum = 0.0 + 0.0im
  for op11 in op1.op
    product = op11[1]
    t2 = copy(t)
    for op12 in op11[2]
      t2 = op12(t2)[2]
      product *= op12(t2)[1]
    end
    if t2 != t1
      throw(ArgumentError("The operator does not preserve the state."))
    end
    sum += product
  end
  return (sum, t1)
end
function -(op1::Op)
  (-1.0 + 0.0im) * op1
end
function -(op1::Op, op2::Op)
  op1 + (-op2)
end
function -(op1::Op, op2::Op...)
  op3 = +(op2...)
  op1 - op3
end
function sum(mats::Op...)
  ans = mats[1]
  for mat in mats[2:end]
    ans += mat
  end
  ans
end
function sum(f::Function, k::Int=0)
  sum(Tuple(f(i) for i in k:(_site[]-1))...)
end
function id(i::Int)
  if i < 1 || i > _dim[]^_site[]
    throw(ArgumentError("i must be in the range [1, dim^site]"))
  end
  return (1.0 + 0.0im, i)
end
id() = Op(id)
function shift(k::Int=1)
  function shift1(t::Int)
    n = Nary_trans(t)
    n1 = circshift(n, k)
    return (1.0 + 0.0im, Nary_reverse(n1))
  end
  return Op(shift1)
end
function site_flip(t::Int)
  n = Nary_trans(t)
  n1 = reverse(n)
  return (1.0 + 0.0im, Nary_reverse(n1))
end
site_flip() = Op(site_flip)
function spin_flip(t::Int)
  n = Nary_trans(t)
  n1 = Vector{Int}(undef, _site[])
  for i in 1:_site[]
    n1[i] = _dim[]/2.0 - n[i]
  end
  return (1.0 + 0.0im, Nary_reverse(n1))
end
spin_flip() = Op(spin_flip)
function spin(kind::Char, site::Int)
  site_number = _site[]
  idx = mod(site-1,site_number)+1
  if kind == '+'
    plus = function (t::Int)
      n = Nary_trans(t)
      n[idx] += 1
      return n[idx] < _dim[] ? (1.0 + 0im, Nary_reverse(n)) : (0.0 + 0im, t)
    end
    return Op(plus)
  elseif kind == '-'
    minus = function (t::Int)
      n = Nary_trans(t)
      n[idx] -= 1
      return n[idx] > -1 ? (1.0 + 0im, Nary_reverse(n)) : (0.0 + 0im, t)
    end
    return Op(minus)
  elseif kind == 'x'
    plus = function (t::Int)
      n = Nary_trans(t)
      n[idx] += 1
      n[idx] < _dim[] ? (1.0 + 0im, Nary_reverse(n)) : (0.0 + 0im, t)
    end
    minus = function (t::Int)
      n = Nary_trans(t)
      n[idx] -= 1
      n[idx] > -1 ? (1.0 + 0im, Nary_reverse(n)) : (0.0 + 0im, t)
    end
    return 0.5 * (Op(plus) + Op(minus))
  elseif kind == 'y'
    plus = function (t::Int)
      n = Nary_trans(t)
      n[idx] += 1
      n[idx] < _dim[] ? (1.0 + 0im, Nary_reverse(n)) : (0.0 + 0im, t)
    end
    minus = function (t::Int)
      n = Nary_trans(t)
      n[idx] -= 1
      n[idx] > -1 ? (1.0 + 0im, Nary_reverse(n)) : (0.0 + 0im, t)
    end
    return (-0.5im) * (Op(plus) - Op(minus))
  elseif kind == 'z'
    z = function (t::Int)
      n = Nary_trans(t)
      ((_dim[] - 1.0) / 2.0 - n[idx] + 0.0im, t)
    end
    return Op(z)
  else
    throw(ArgumentError("kind must be '+', '-', 'x', 'y', or 'z'"))
  end
end
function num(site::Int)
  number = function (t::Int)
    n = Nary_trans(t)
    return (n[site] + 0.0im, t)
  end
  return Op(number)
end
function S_z()
  return sum(j->spin('z', j))
end
function matrix(op1::Union{Matrix{ComplexF64},Op})
  if isa(op1, Matrix{ComplexF64})
    return op1
  end
  dimention = _dim[]
  site_number = _site[]
  dim_tot = dimention^site_number
  mat = zeros(ComplexF64, dim_tot, dim_tot)
  for t in 1:dim_tot
    for (coeff, op11) in op1.op
      t1 = t
      coeff1 = coeff
      for op12 in op11
        (v1, t1) = op12(t1)
        coeff1 *= v1
      end
      mat[t, t1] += coeff1
    end
  end
  return mat
end
function complex_formatter(; digits::Int=1)
  return (v, i, j) -> begin
    if v == 0 + 0im
      @sprintf("%.*f", digits, 0.0)
    elseif isa(v, Complex)
      rea = round(real(v), digits=digits)
      image = round(imag(v), digits=digits)
      if image == 0
        @sprintf("%.*f", digits, rea)
      elseif rea == 0
        @sprintf("%.*fim", digits, image)
      else
        sign = image > 0 ? "+" : "-"
        @sprintf("%.*f%s%.*fim", digits, rea, sign, digits, abs(image))
      end
    elseif isa(v, Number)
      @sprintf("%.*f", digits, v)
    else
      string(v)
    end
  end
end
function show(op1::Op, digit::Int=1)
  mat1 = matrix(op1)
  pretty_table(mat1, header=([join(string.(Nary_trans(i)), "") for i in 1:size(mat1, 1)]), row_labels=([join(string.(Nary_trans(i)), "") for i in 1:size(mat1, 2)]), formatters=complex_formatter(digits=digit))
end
function show_eigens(vec1::Vector{Float64},mat1::Matrix{ComplexF64}, digit::Int=1)
  mat2=transpose(mat1)
  pretty_table(mat2, header=([join(string.(Nary_trans(i)), "") for i in 1:size(mat1, 1)]), row_labels=([string(vec1[i]) for i in 1:size(mat1, 2)]), formatters=complex_formatter(digits=digit))
end
function nthroots(z::ComplexF64, n::Int)
  if n != 1 && abs(imag(z)) < 1.0e-10 && real(z) < 1.0e-10
    throw(ArgumentError("n must be a positive integer greater than 0."))
  end
    r = abs(z)
  θ = angle(z)
  roots = Vector{ComplexF64}(undef, n)
  for k in 1:n
    if n / k == 2 && abs(imag(z)) < 1.0e-10 && real(z) > 1.0e-10
      roots[k] = -sqrt(real(z)) + 0.0im
      continue
    end
    if n == k
      roots[k] = r^(1 / n) * cis(θ / n)
      continue
    end
    roots[k] = r^(1 / n) * cis(θ / n + 2π * ((n / k)^(-1)))
  end
  return roots
end
function eigens_dict(op1::Op)
  dim_tot = _dim[]^_site[]
  dict1 = Dict{ComplexF64,Vector{Vector{Int}}}()
  set1 = Set{Int}()
  maximum = 0
  for i in 1:dim_tot
    if i in set1
      continue
    end
    i1 = copy(i)
    vec1 = Vector{Int}()
    push!(vec1, i1)
    push!(set1, i1)
    ntheigenvalue = 1.0 + 0.0im
    for j in 1:dim_tot
      tup1 = op1 * i1
      ntheigenvalue *= tup1[1]
      if tup1[2] == i
        maximum = max(maximum, j)
        break
      end
      i1 = copy(tup1[2])
      push!(vec1, i1)
      push!(set1, i1)
    end
    roots1= nthroots(ntheigenvalue, length(vec1))
    for eigenvalue in roots1
      dict1[eigenvalue] = get(dict1, eigenvalue, Vector{Vector{Int}}())
      push!(dict1[eigenvalue], vec1)
    end
  end
  return (dict1, maximum)
end
function eigen_number(eigenvalue::ComplexF64, maximum::Int)
  root1=nthroots(1.0 + 0.0im, maximum)
  number = 0
  for i in eachindex(root1)
    if abs(root1[i] - eigenvalue) < 1.0e-5
      number = i
      break
    end
  end
  return number
end
function matrix(ham::Union{Matrix{ComplexF64},Op}, eigens_vec::Vector{Vector{Int}}, maximum::Int, eigenvalue::ComplexF64, number::Int)
  if isa(ham, Matrix{ComplexF64})
    mat2 = ham
  else
    mat2 = matrix(ham)
  end
  mat1 = zeros(ComplexF64, length(eigens_vec), length(eigens_vec))
  roots1 = nthroots(1.0 + 0.0im, maximum)
  for i in eachindex(eigens_vec)
    len_n= length(eigens_vec[i])
    for j in eachindex(eigens_vec)
      len_l= length(eigens_vec[j])
      s1= 0.0 + 0.0im
      for k in eachindex(eigens_vec[j])
        s1+=(sqrt(len_n)/sqrt(len_l))roots1[mod(number*(k-1)-1, maximum) + 1]*mat2[eigens_vec[j][k],eigens_vec[i][1]]
      end
      mat1[j,i] += s1
    end
  end
  return mat1
end
function reverse_eigenstate(vec1::Vector{ComplexF64},eigens_vec::Vector{Vector{Int}}, maximum::Int, eigenvalue::ComplexF64, number::Int)
  dim_tot = _dim[]^_site[]
  vec2=zeros(ComplexF64, dim_tot)
  for i in eachindex(eigens_vec)
    len_vec= length(eigens_vec[i])
    roots1 = nthroots(1.0 + 0.0im, len_vec)
    for j in eachindex(eigens_vec[i])
      vec2[eigens_vec[i][j]] += (1.0/sqrt(length(eigens_vec[i]))) * roots1[mod(-number*(j-1)-1, len_vec) + 1] * vec1[i]
    end
  end
  return vec2
end
function reverse_eigenstates(eigenstates::Matrix{ComplexF64},eigens_vec::Vector{Vector{Int}}, maximum::Int, eigenvalue::ComplexF64, number::Int)
  dim_tot = _dim[]^_site[]
  eigenstates1=Matrix{ComplexF64}(undef, dim_tot, length(eigens_vec))
  for i in eachindex(eigens_vec)
    eigenstates1[:,i] = reverse_eigenstate(eigenstates[:,i], eigens_vec, maximum, eigenvalue, number)
  end
  return eigenstates1
end
function complex_float(x::Union{ComplexF64,Float64})
  if isa(x, Float64)
    return x
  elseif abs(imag(x)) < 1.0e-10
    return real(x)
  else
    throw(ArgumentError("x must be a real number or a complex number with negligible imaginary part."))
  end
end
function complex_float(vec::Union{Vector{ComplexF64},Vector{Float64}})
  vec1 = Vector{ComplexF64}(undef, length(vec))
  for i in eachindex(vec)
    vec1[i] = complex_float(vec[i])
  end
  return vec1
end
function block_diag1(ham::Op, op1::Op, b::Bool=false)
  dim_tot = _dim[]^_site[]
  mat1=matrix(ham)
  dict1, maximum = eigens_dict(op1)
  sorted_keys = sort(collect(keys(dict1)), by=real)
  eigenenergys1 = Vector{Vector{Float64}}(undef, length(sorted_keys))
  eigenstates1 = Vector{Matrix{ComplexF64}}(undef, length(sorted_keys))
  for i in eachindex(sorted_keys)
    eigenvalue1 = sorted_keys[i]
    vec1 = dict1[eigenvalue1]
    number=eigen_number(eigenvalue1, maximum)
    mat2=matrix(mat1,vec1, maximum, eigenvalue1, number)
    eigenvalues1, eigenvectors1=eigen(mat2)
    eigenenergys1[i]= complex_float(eigenvalues1)
    eigenstates1[i]=reverse_eigenstates(eigenvectors1, vec1, maximum, eigenvalue1, number)
  end
  if b
    return (sorted_keys, eigenenergys1, eigenstates1)
  end
  return (vcat(eigenenergys1...), hcat(eigenstates1...))
end
function energy_same_check(energy1::Vector{Float64}, energy2::Vector{Float64})
  if length(energy1) != length(energy2)
    throw(ArgumentError("The lengths of the two energy vectors are not equal."))
  end
  sorted_energy1 = sort(energy1)
  sorted_energy2 = sort(energy2)
  for i in 1:length(energy1)
    if abs(sorted_energy1[i] - sorted_energy2[i]) > 1.0e-10
      return false
    end
  end
  return true
end
function block_diag1_eigenvalue_energy(ham::Op, op1::Op)
  dim_tot = _dim[]^_site[]
  mat1=matrix(ham)
  dict1, maximum = eigens_dict(op1)
  sorted_keys = sort(collect(keys(dict1)), by=real)
  eigenenergys1 = [ Vector{Float64}(undef, length(sorted_keys)) for _ in 1:length(sorted_keys) ]
  for i in eachindex(sorted_keys)
    eigenvalue1 = sorted_keys[i]
    vec1 = dict1[eigenvalue1]
    number=eigen_number(eigenvalue1, maximum)
    mat2=matrix(mat1,vec1, maximum, eigenvalue1, number)
    eigenvalues1, _=eigen(mat2)
    eigenenergys1[i]= complex_float(eigenvalues1)
  end
  return (sorted_keys,eigenenergys1)
end
function rvalue(vec::Vector{Float64})
  if length(vec) < 3
    return -1.0
  end
  sorted_vec = sort(vec)
  r = 0.0
  for i in 1:length(sorted_vec)-2
    min_diff = min(sorted_vec[i+1] - sorted_vec[i], sorted_vec[i+2] - sorted_vec[i+1])
    max_diff = max(sorted_vec[i+1] - sorted_vec[i], sorted_vec[i+2] - sorted_vec[i+1])
    r += min_diff / max_diff
  end
  return r / (length(sorted_vec) - 2)
end
function rvalue_modify(vec::Vector{Float64})
  if length(vec) < 8
    return -1.0
  end
  sorted_vec = sort(vec)
  sorted_delted_vec=sorted_vec[floor(Int,length(sorted_vec)/4.0):end-floor(Int,length(sorted_vec)/4.0)]
  r = 0.0
  for i in 1:length(sorted_delted_vec)-2
    min_diff = min(sorted_delted_vec[i+1] - sorted_delted_vec[i], sorted_delted_vec[i+2] - sorted_delted_vec[i+1])
    max_diff = max(sorted_delted_vec[i+1] - sorted_delted_vec[i], sorted_delted_vec[i+2] - sorted_delted_vec[i+1])
    r += min_diff / max_diff
  end
  return r / (length(sorted_delted_vec) - 2)
end
function block_diag1_rvalue(ham::Op, op1::Op, modify::Bool=false)
  sorted_keys,eigenenergys1=block_diag1_eigenvalue_energy(ham, op1)
  if length(sorted_keys)!= length(eigenenergys1)
    throw(ArgumentError("The lengths of the sorted keys and eigen energies do not match."))
  end
  dict1= Dict{ComplexF64,Float64}()
  for i in eachindex(sorted_keys)
    eigenvalue1 = sorted_keys[i]
    if modify
      r = rvalue_modify(eigenenergys1[i])
    else
      r = rvalue(eigenenergys1[i])
    end
    dict1[eigenvalue1] = r
  end
  return dict1
end
function block_diag1_rvalue(ham::Op, op1::Op, eigenvalue::ComplexF64, modify::Bool=false)
  sorted_keys,eigenenergys1=block_diag1_eigenvalue_energy(ham, op1)
  if length(sorted_keys)!= length(eigenenergys1)
    throw(ArgumentError("The lengths of the sorted keys and eigen energies do not match."))
  end
  dict1= Dict{ComplexF64,Float64}()
  for i in eachindex(sorted_keys)
    eigenvalue1 = sorted_keys[i]
    if abs(eigenvalue1-eigenvalue) < 1.0e-10
      if modify
        return rvalue_modify(eigenenergys1[i])
      else
        return rvalue(eigenenergys1[i])
      end
    end
  end
  throw(ArgumentError("The specified eigenvalue is not found in the sorted keys."))
end
function entanglment_entropy(energyvector::Vector{ComplexF64}, nsite::Int=floor(Int, _site[] / 2))
  if length(energyvector) != _dim[]^_site[]
    throw(ArgumentError("The length of energyvector must be equal to dim^site."))
  end
  mat = zeros(ComplexF64, _dim[]^nsite, _dim[]^(_site[] - nsite))
  for i in 1:_dim[]^nsite
    for j in 1:_dim[]^(_site[]-nsite)
      mat[i, j] = energyvector[(i-1)*(_dim[]^(_site[]-nsite))+j]
    end
  end
  mat_S = svd(mat).S
  ans = 0.0
  for i in eachindex(mat_S)
    ans += -abs(mat_S[i])^2 * log(abs(mat_S[i])^2)
  end
  return ans
end
function entanglement_entropies(energyvector::Matrix{ComplexF64}, nsite::Int=floor(Int, _site[] / 2))
  vector1 = Vector{Float64}(undef, size(energyvector,2))
  for i in 1:size(energyvector,2)
    energyvector1 = energyvector[:, i]
    entanglment_entropy1 = entanglment_entropy(energyvector1, nsite)
    vector1[i] = entanglment_entropy1
  end
  return vector1
end
function block_diag1_entanglement_entropy(ham::Op, op1::Op, nsite::Int=floor(Int, _site[] / 2))
  sorted_keys, eigenenergies1,eigenstates1=block_diag1(ham, op1, true)
  dict1= Dict{ComplexF64,Tuple{Vector{Float64},Vector{Float64}}}()
  if length(eigenenergies1) != length(eigenstates1)&& length(eigenenergies1) != length(sorted_keys)
    throw(ArgumentError("The lengths of eigenenergies and eigenstates do not match."))
  end
  for i in eachindex(eigenenergies1)
    vector1=entanglement_entropies(eigenstates1[i], nsite)
    dict1[sorted_keys[i]] = (eigenenergies1[i], vector1)
  end
end
function block_diag1_entanglement_entropy(ham::Op, op1::Op, eigenvalue::ComplexF64, nsite::Int=floor(Int, _site[] / 2))
  sorted_keys, eigenenergies1,eigenstates1=block_diag1(ham, op1, true)
  dict1= Dict{ComplexF64,Tuple{Vector{Float64},Vector{Float64}}}()
  if length(eigenenergies1) != length(eigenstates1)&& length(eigenenergies1) != length(sorted_keys)
    throw(ArgumentError("The lengths of eigenenergies and eigenstates do not match."))
  end
  for i in eachindex(eigenenergies1)
    vector1=entanglement_entropies(eigenstates1[i], nsite)
    if abs(eigenvalue-sorted_keys[i]) < 1.0e-10
      return (eigenenergies1[i], vector1)
    end
  end
  throw(ArgumentError("The specified eigenvalue is not found in the sorted keys."))
end
end

Main.ExactDiag

In [ ]:
using .ExactDiag
L = 8
init(2,L)
hj = sqrt(1.5)
vj = sqrt(2.0)
Δ = sqrt(1.3)
kj = sqrt(2.5)
H1 = sum(j -> spin('z', j) * spin('z', j + 1), 1) - sum(j -> hj * spin('x', j)) - sum(j -> vj * spin('z', j))
H2 = sum(j -> spin('z', j) * spin('z', j + 1), 2) + kj * spin('z', 1) * spin('z', 2) - sum(j -> hj * spin('x', j))
H3 = sum(j -> spin('x', j) * spin('x', j + 1) + spin('y', j) * spin('y', j + 1) + Δ * spin('z', j) * spin('z', j + 1),1) - sum(j -> vj * sin(j) * spin('z', j))
H4 = sum(j -> spin('z', j) * spin('z', j + 1), 1) - sum(j -> 10000.0 * spin('x', j)) - sum(j -> vj * spin('z', j))

Op(Tuple{ComplexF64, Vector{Function}}[(1.0 + 0.0im, [ExactDiag.var"#14#21"{Int64}(1), ExactDiag.var"#14#21"{Int64}(2)]), (1.0 + 0.0im, [ExactDiag.var"#14#21"{Int64}(2), ExactDiag.var"#14#21"{Int64}(3)]), (1.0 + 0.0im, [ExactDiag.var"#14#21"{Int64}(3), ExactDiag.var"#14#21"{Int64}(4)]), (1.0 + 0.0im, [ExactDiag.var"#14#21"{Int64}(4), ExactDiag.var"#14#21"{Int64}(5)]), (1.0 + 0.0im, [ExactDiag.var"#14#21"{Int64}(5), ExactDiag.var"#14#21"{Int64}(6)]), (1.0 + 0.0im, [ExactDiag.var"#14#21"{Int64}(6), ExactDiag.var"#14#21"{Int64}(7)]), (1.0 + 0.0im, [ExactDiag.var"#14#21"{Int64}(7), ExactDiag.var"#14#21"{Int64}(8)]), (-5000.0 + 0.0im, [ExactDiag.var"#10#17"{Int64}(8)]), (-5000.0 + 0.0im, [ExactDiag.var"#11#18"{Int64}(8)]), (-5000.0 + 0.0im, [ExactDiag.var"#10#17"{Int64}(1)])  …  (-5000.0 + 0.0im, [ExactDiag.var"#10#17"{Int64}(7)]), (-5000.0 + 0.0im, [ExactDiag.var"#11#18"{Int64}(7)]), (-1.4142135623730951 + 0.0im, [ExactDiag.var"#14#21"{Int64}(8)]), (-1.4142135623730951 + 0.0im, [ExactDiag.

In [6]:
using ExactDiag
using Plots
default(dpi=600)
x1=Vector{Float64}()
x2=Vector{Float64}()
x3=Vector{Float64}()
x4=Vector{Float64}()
y1=Vector{Float64}()
y2=Vector{Float64}()
y3=Vector{Float64}()
y4=Vector{Float64}()
L = 2
init(2,L)
hj = 1.0
vj = 1.0
Δ = sqrt(1.3)
kj = sqrt(2.5)
H1 = sum(j -> spin('z', j) * spin('z', j + 1), 1) - sum(j -> hj * spin('x', j)) - sum(j -> vj * spin('z', j))
# H2 = sum(j -> spin('z', j) * spin('z', j + 1), 2) + kj * spin('z', 1) * spin('z', 2) - sum(j -> hj * spin('x', j))
H3 = sum(j -> spin('x', j) * spin('x', j + 1) + spin('y', j) * spin('y', j + 1) + Δ * spin('z', j) * spin('z', j + 1),1) - sum(j -> vj * sin(j) * spin('z', j))
H4 = sum(j -> spin('z', j) * spin('z', j + 1), 1) - sum(j -> 10000.0 * spin('x', j)) - sum(j -> vj * spin('z', j))
x1, y1 = block_diag1_entanglement_entropy(H1, site_flip(), 1.0 + 0.0im)
#x2, y2 = block_diag1_entanglement_entropy(H2, spin_flip(), 1.0 + 0.0im)
x3,y3 = block_diag1_entanglement_entropy(H3, S_z(), 0.0 + 0.0im)
x4,y4 = block_diag1_entanglement_entropy(H4, site_flip(), 1.0 + 0.0im)
println(x1,y1)

[-1.3206680578276817, 0.0075690235640352614, 1.563099034263647][0.024579379606368943, 0.6333266223788279, 0.007530978175650264]


In [3]:
using .ExactDiag
using Plots
default(dpi=600)
x=Vector{Float64}()
y11=Vector{Float64}()
y12=Vector{Float64}()
y21=Vector{Float64}()
y22=Vector{Float64}()
for L in 5:10
  push!(x, L)
  init(2,L)
  hj = sqrt(1.5)
  vj = sqrt(2.0)
  Δ = sqrt(1.3)
  H1 = sum(j -> spin('z', j) * spin('z', j + 1), 1) - sum(j -> hj * spin('x', j)) - sum(j -> vj * spin('z', j))
  H2 = sum(j -> spin('x', j) * spin('x', j + 1) + spin('y', j) * spin('y', j + 1) + Δ * spin('z', j) * spin('z', j + 1),1) - sum(j -> hj * spin('x', j)) - sum(j -> vj * spin('z', j))
  dict1= block_diag1_rvalue(H1, site_flip(), true)
  push!(y11, dict1[-1.0 + 0.0im])
  push!(y12, dict1[1.0 + 0.0im])
  dict2= block_diag1_rvalue(H2, site_flip(), true)
  push!(y21, dict2[-1.0 + 0.0im])
  push!(y22, dict2[1.0 + 0.0im])
end
plot(x, y11, label="eigenvalue=-1.0", xlabel="site number", ylabel="rvalue", title="Transverse magnetic field Ising model with longitudinal magnetic field", legend=:topright, marker=:circle, markersize=5, linewidth=2, color=:blue)
plot!(x, y12, label="eigenvalue=1.0",marker=:diamond, markersize=5, linewidth=2, color=:green)
threshold = 4.0-2.0*sqrt(3.0)
hline!([threshold];
  label="GOE",
  linestyle=:dash, linewidth=2, color=:red)
cd(raw"\\wsl.localhost\Ubuntu\home\kokor\git\exact_diag")
savefig("rvalue_transverse1.png")
plot(x, y21, label="eigenvalue=-1.0", xlabel="site number", ylabel="rvalue", title="XXZ model with transverse and longitudinal magnetic field", legend=:topright, marker=:circle, markersize=5, linewidth=2, color=:blue)
plot!(x, y22, label="eigenvalue=1.0",marker=:diamond, markersize=5, linewidth=2, color=:green)
threshold = 4.0-2.0*sqrt(3.0)
hline!([threshold];
  label="GOE",
  linestyle=:dash, linewidth=2, color=:red)
cd(raw"\\wsl.localhost\Ubuntu\home\kokor\git\exact_diag")
savefig("rvalue_xxz1.png")

"\\\\wsl.localhost\\Ubuntu\\home\\kokor\\git\\exact_diag\\rvalue_xxz1.png"

現在の課題、横磁場縦磁場イジングモデルにおけるrvalueの値が小さい

In [18]:
using .ExactDiag
using LinearAlgebra
init(2,12)
hj = sqrt(1.5)
vj = sqrt(2.0)
Δ = sqrt(1.3)
Uj = 2.0
H1 = sum(j -> spin('z', j) * spin('z', j + 1), 1) - sum(j -> hj * spin('x', j)) - sum(j -> vj * spin('z', j))
H2 = sum(j -> spin('x', j) * spin('x', j + 1) + spin('y', j) * spin('y', j + 1) + Δ * spin('z', j) * spin('z', j + 1),1) - sum(j -> hj * spin('x', j)) - sum(j -> vj * spin('z', j))
H3 = sum(j -> -spin('+', j) * spin('-', j + 1) - spin('-', j) * spin('+', j + 1), 1) + sum(j -> (Uj / 2.0) * num(j) * (num(j) - 1.0*id()))
println(block_diag1_rvalue(H2, site_flip(),true))

Dict{ComplexF64, Float64}(-1.0 + 0.0im => 0.4850118599316128, 1.0 + 0.0im => 0.4690404025322195)
